# Imports 

In [11]:
# conda env create -f env.yaml 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import os

# Data setup

In [12]:
np.random.seed(0)
os.environ["OMP_NUM_THREADS"] = "5"

df3 = pd.read_csv("Data/data3.csv")
df3

,failures,higher,studytime,Medu,Fedu,Dalc,age,reason_reputation,school,address,internet,G3
0,0,1,2,4,4,1,18,0,0,0,0,6
1,0,1,2,1,1,1,17,0,0,0,1,6
2,3,1,2,1,1,2,15,0,0,0,1,10
3,0,1,3,4,2,1,15,0,0,0,1,15
4,0,1,2,3,3,1,16,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...
1039,1,1,3,2,3,1,19,0,1,1,1,10
1040,0,1,2,3,1,1,18,0,1,0,1,16
1041,0,1,2,1,1,1,18,0,1,0,0,9
1042,0,1,1,3,1,3,17,0,1,0,1,10


In [106]:
df3_copy = df3.copy()
y = df3_copy.pop("G3")
y_binary = [0 if y_i > np.mean(y) else 1 for y_i in y] #making the target class into binary (over/under average score)
X = df3_copy

In [107]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)
X_train.shape

(835, 11)

Note: No need to scale, trees are not affected by different scales.

# Tree 1

In [123]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import randint

In [166]:
parameters = {"max_depth": [3, None],
              "max_features": randint(1, X.shape[1]),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}

clf = tree.DecisionTreeClassifier()
tree_cv = RandomizedSearchCV(clf, parameters, cv=10, random_state = 10)
tree_cv.fit(X_train,y_train)

RandomizedSearchCV(cv=10, estimator=DecisionTreeClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, None],
                                        'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E4ABF87760>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E4AC0ACD60>},
                   random_state=10)

# Evaluation

#### Training results

In [167]:
print("Best Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

Best Decision Tree Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': 7, 'min_samples_leaf': 7}
Best score is 0.6672690763052209


In [168]:
tree_cv.cv_results_

{'mean_fit_time': array([0.00260098, 0.00240726, 0.00190086, 0.00180228, 0.00179989,
        0.00180004, 0.00189826, 0.00200558, 0.00199864, 0.00210238]),
 'std_fit_time': array([8.00565241e-04, 4.91354984e-04, 3.00075291e-04, 3.94843742e-04,
        4.00042818e-04, 3.99698213e-04, 3.07554043e-04, 1.18208499e-05,
        4.91680156e-06, 2.99334673e-04]),
 'mean_score_time': array([0.00149887, 0.00099761, 0.00119939, 0.00130026, 0.00119987,
        0.00120013, 0.00110176, 0.00129161, 0.00150075, 0.00120029]),
 'std_score_time': array([4.99184991e-04, 5.51571992e-06, 3.99936861e-04, 4.58510803e-04,
        4.00054784e-04, 3.99507080e-04, 3.07633045e-04, 4.60583268e-04,
        5.00930544e-04, 4.00973940e-04]),
 'param_criterion': masked_array(data=['entropy', 'gini', 'gini', 'gini', 'gini', 'gini',
                    'entropy', 'gini', 'gini', 'entropy'],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?'

In [169]:
print(f"Mean test scores: ", tree_cv.cv_results_["mean_test_score"])

Mean test scores:  [0.64443488 0.63136833 0.63245841 0.62527252 0.63356282 0.64559667
 0.60130522 0.60971027 0.64568273 0.66726908]


#### Test set results

In [170]:
#cross validation

CV_test = cross_val_score(tree_cv, X_test, y_test, cv=10)
print(CV_test)
print("Mean test accuracy: ",np.mean(CV_test))

[0.61904762 0.52380952 0.61904762 0.57142857 0.61904762 0.47619048
 0.80952381 0.42857143 0.61904762 0.6       ]
Mean test accuracy:  0.5885714285714284


In [174]:
#overall test
print("Training accuracy: ",tree_cv.score(X_train, y_train))
print("Test accuracy: ", tree_cv.score(X_test, y_test))

Training accuracy:  0.7485029940119761
Test accuracy:  0.6220095693779905


# Tree 2

TODO

# Tree 3

TODO